In [1]:
import sys, os, re, json
from copy import deepcopy
import cobra
import pandas as pd
from cobra.core.dictlist import DictList
from sumoflux.settings import MODEL_DIR
import numpy as np
from cobra import Reaction
display = pd.options.display
display.max_columns = 200
display.max_rows = 300
display.max_colwidth = 199
display.width = None

In [2]:
core_file = os.path.join(MODEL_DIR, 'sbml', 'e_coli_core.xml')
ijo_file = os.path.join(MODEL_DIR, 'sbml', 'iJO1366.xml')
bwg_file = os.path.join(MODEL_DIR, 'sbml', 'iBWG_1329.xml')
b21_file = os.path.join(MODEL_DIR, 'sbml', 'iB21_1397.xml')

ijo = cobra.io.read_sbml_model(ijo_file)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-12


In [3]:
core = cobra.io.read_sbml_model(core_file)
metabolites_to_add = [
    # Entner-Doudoroff 
    '2ddg6p_c',
    
    # glycerol uptake 
    'glyc_e',
    'glyc_c',
    'glyc3p_c',
    'dha_c',
    'dha_e',
    
    # xylose uptake
    'xyl__D_e',
    'xyl__D_c',
    'xylu__D_c',
    
    # sulfate uptake
    'so4_e',
    'so4_c',
]

core.add_metabolites(metabolite_list=[ijo.metabolites.get_by_id(mid).copy() for mid in metabolites_to_add])

reactions_to_add0 = []
for rid in ['GLYCtpp', 'DHAtpp', 'XYLabcpp', 'XYLt2pp', 'SO4t2pp', 'SULabcpp']:
    reaction = ijo.reactions.get_by_id(rid).copy()
    reaction.id = re.sub('pp$', '', rid)
    for m in reaction.metabolites:
        if m.compartment == 'p':
            m.compartment = 'e'
            met_e = core.metabolites.get_by_id(m.id.replace('_p', '_e'))
            reaction._metabolites[met_e] = reaction._metabolites.pop(m)
            if ('C' in met_e.elements) and (met_e in reaction.products):
                reaction *= -1
    reaction = reaction.copy() # otherwise the metabolite.reactions attribute is all kinds of fucked
    reactions_to_add0.append(reaction)
    
reactions_to_add = [
    'EDD', 'EDA', 
    'EX_glyc_e', 'GLYK', 'G3PD5', 'G3PD2', 'F6PA', 'GLYCDx', 'DHAPT', 'EX_dha_e', 
    'EX_xyl__D_e', 'XYLI1', 'XYLK', 
    'EX_so4_e', 
]


core.add_reactions(reaction_list=[ijo.reactions.get_by_id(rid).copy() for rid in reactions_to_add] + reactions_to_add0)

In [4]:
import_reaction_ids = {
    'CO2t': 1, 
    'GLCpts': 1, 
    'FRUpts2': 1, 
    'XYLt2': 1,
    'XYLabc': 1,
    'SUCCt2_2': 1,
    'SUCCt3': 1,
    'PYRt2': 1,
    'ACt2r': 1,
    'D_LACt2': 1,
    'ETOHt2r': 1,
    'ACALDt': 1,
    'FORt': -1,
    'FORt2': 1,
    'D_LACt2': 1,
    'AKGt2r': 1,
    'MALt2_2': 1,
    'FUMt2_2': 1,
    'GLNabc': 1, 
    'GLUt2r': 1,
    'GLYCt': -1,
    'DHAt': 1,
}

for rid, coef in import_reaction_ids.items():
    core.reactions.get_by_id(rid).bounds = (0.0,0.0)

co2 = core.reactions.get_by_id('CO2t')
co2.bounds = (0.0, 1000.0)
co2_out = Reaction(id='OUT_CO2t', lower_bound=0.0, upper_bound=1000.0)
co2_out.add_metabolites({core.metabolites.get_by_id('co2_c'): -1})
core.add_reactions(reaction_list=[co2_out])



for reac in core.boundary:
    if reac.id == 'OUT_CO2t':
        continue
    if reac.id == 'EX_co2_e':
        reac.upper_bound = 0.0
    else:
        for m in reac.metabolites:
            if 'C' in m.elements:
                reac.bounds = (0.0, 0.0)

In [5]:
# these bounds are glucose growth!

# ex = core.reactions.get_by_id('EX_glc__D_e')
# ex.bounds = (-10.0, -10.0)
# pts = core.reactions.get_by_id('GLCpts')
# pts.bounds = (10.0, 10.0)

bm = core.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
akg = core.metabolites.get_by_id('akg_c')
glu = core.metabolites.get_by_id('glu__L_c')
bm -= core.reactions.get_by_id('GLUDy') * bm.metabolites[akg]
bm._metabolites[glu] = round(bm.metabolites[glu], 4)

In [6]:
core.objective = {bm: 1}
solution = core.optimize()
summ = core.summary()
summ

C:\Miniconda3\envs\pta\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Infeasible: None (infeasible).

In [7]:
sbml_file =   os.path.join(MODEL_DIR, 'sbml', 'e_coli_core_tomek.xml')
json_file =   os.path.join(MODEL_DIR, 'escher_input', 'model', 'e_coli_core_tomek.json')
escher_file = os.path.join(MODEL_DIR, 'escher_input', 'map',   'e_coli_core_tomek.json')

cobra.io.write_sbml_model(core, sbml_file)
cobra.io.save_json_model(core, json_file)

IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      21.8   h2o_e  29.2   Biomass_Ecol...  0.874
glc__D_e  10     co2_e  22.8
nh4_e      4.77  h_e    17.5
pi_e       3.21

In [23]:
# from cobra.flux_analysis import flux_variability_analysis
# fvadf = flux_variability_analysis(core)
# fvadf

In [24]:
import json
import escher
from escher import Builder

builder = Builder(
    model_json=json_file,
    map_json=escher_file,
    reaction_data=solution.fluxes,
)
builder.save_html('e_coli_core_tomek.html')